# Recipe Generation and Similarity Matching

This notebook demonstrates how to generate recipes based on a shopping cart and compare them against an indexed recipe database to find similar existing recipes.

## Setup and Imports

Import required libraries for AI content generation, data processing, and vector search operations.

In [ ]:
from google import genai
from google.genai.types import EmbedContentConfig
import json
import pandas as pd

## Configuration and Client Setup

Configure project settings and initialize the Gemini AI client for content generation.

In [43]:
import os

index_endpoint = "3153243217810423808" # @param

PROJECT_ID = "sandbox-401718"  # @param {type:"string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Input Data: Shopping Cart

Load the sample shopping cart containing ingredients that will be used for recipe generation.

In [4]:
# Sample Input 
### ['chicken breasts', 'broccoli', 'carrots', 'onion']

with open('sample_cart.txt', 'r') as file:
    cart_list = json.load(file)['cart_list']

# cart list
print(cart_list)

['chicken breasts', 'broccoli', 'carrots', 'onion']


## Recipe Generation Prompt

Create a comprehensive prompt for the AI model to generate personalized recipe recommendations based on the shopping cart items.

In [5]:
prompt =f"""
**System Instruction**

You are a helpful AI assistant specialized in providing personalized recipe recommendations. Your primary function is to act as a **Recipe Assistant** that tailors meal plans and recipes to a user's available groceries.

**Your Task:**
1. Silently review the user's `cart_list`.
2. Identify 1-2 recipes that maximize the use of ingredients from the `cart_list`.
3. If necessary, supplement with common ingredients from the `recommendation_list` below to complete the recipes.
4. Prioritize recipes that use the highest number of items from the `cart_list`.
5. Generate a JSON response for each recipe according to the specified schema.

**Inputs:**
*  **cart_list:** A list of grocery items provided by the user.
  `\n\n{cart_list}\n\n`

*  **recommendation_list:** A predefined list of common pantry staples.
  ```
  [
   "milk", "eggs", "bread", "butter", "cheese", "all-purpose flour",
   "rice", "pasta", "chicken", "beef", "onions", "garlic",
   "potatoes", "tomatoes", "canned tomatoes", "beans", "olive oil",
   "salt", "pepper", "sugar"
  ]
  ```

**Desired JSON Schema and Description:**

*  **`title`**: (String) The descriptive name of the dish.
*  **`estimated_cooking_time`**: (String) The total time required for preparation and cooking.
*  **`ingredients_from_cart`**: (Array of Objects) A list of ingredients required for the recipe that are present in the `cart_list`. Each object should contain:
  *  **`item`**: (String) The name of the ingredient.
  *  **`quantity`**: (String) The specific quantity needed for the recipe.
*  **`ingredients_to_add`**: (Array of Objects) A list of ingredients required for the recipe that are from the `recommendation_list` and not in the `cart_list`. Each object should contain:
  *  **`item`**: (String) The name of the ingredient.
  *  **`quantity`**: (String) The specific quantity needed for the recipe.
*  **`instructions`**: (Array of Strings) Numbered steps for preparing the dish.

**Example of EXACT Desired Output Format:**

```json
[
 {{
  "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
  "estimated_cooking_time": "1 hour 45 minutes",
  "ingredients_from_cart": [
   {{"item": "acorn squash", "quantity": "2 small (about 3 lb. total)"}},
   {{"item": "sage", "quantity": "2 Tbsp. finely chopped"}},
   {{"item": "rosemary", "quantity": "1 Tbsp. finely chopped"}},
   {{"item": "ground allspice", "quantity": "¼ tsp."}},
   {{"item": "crushed red pepper flakes", "quantity": "Pinch"}},
   {{"item": "apples", "quantity": "2 medium (such as Gala or Pink Lady)"}},
   {{"item": "apple cider vinegar", "quantity": "3 Tbsp."}},
   {{"item": "white miso", "quantity": "1 Tbsp. plus 2 tsp."}},
   {{"item": "dry white wine", "quantity": "¼ cup"}},
   {{"item": "unsalted chicken broth", "quantity": "2 cups"}}
  ],
  "ingredients_to_add": [
   {{"item": "whole chicken", "quantity": "1 (3½–4-lb.)"}},
   {{"item": "kosher salt", "quantity": "2¾ tsp., plus more"}},
   {{"item": "unsalted butter", "quantity": "9 Tbsp. total, divided"}},
   {{"item": "black pepper", "quantity": "Freshly ground, to taste"}},
   {{"item": "sturdy white bread", "quantity": "⅓ loaf (about 2½ cups torn)"}},
   {{"item": "extra-virgin olive oil", "quantity": "2 Tbsp."}},
   {{"item": "red onion", "quantity": "½ small, thinly sliced"}},
   {{"item": "all-purpose flour", "quantity": "¼ cup"}}
  ],
  "instructions": [
   "1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.",
   "2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½\"-thick wedges; arrange on a rimmed baking sheet.",
   "3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.",
   "4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.",
   "5. Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.",
   "6. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.",
   "7. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.",
   "8. Using your fingers, mash flour and 2 Tbsp. room-temperature butter in a small bowl to combine.",
   "9. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup. Add wine and cook, scraping up any browned bits, until reduced by about half, about 2 minutes. Add butter mixture; cook, stirring, until a smooth paste forms, about 2 minutes. Add broth and cook, stirring constantly, until thickened, 6–8 minutes. Remove from heat and stir in the remaining miso. Season with salt and black pepper.",
   "10. Serve chicken with gravy and squash panzanella alongside."
  ]
 }}
]
```
"""

In [ ]:
# Generate recipes using Gemini 2.5 Pro
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=[
        prompt,
    ],
)
print(response.text)

## Check if Recipe Generaiton is semantic similar to database of Recipes

### Recipe Processing and Formatting

Parse and format the generated recipes for comparison against the recipe database.

In [8]:
raw_text_output = response.text

# Clean the string to remove the markdown code fences.
clean_json_string = raw_text_output.strip().removeprefix('```json').removesuffix('```').strip()

# Parse the clean JSON string into a Python list of dictionaries.
# Now 'recipes' will be the data structure you expect.
recipes = json.loads(clean_json_string)

# Initialize an empty list to log each processed recipe ---
processed_recipes_list = []

# Now this loop will work perfectly.
# 'recipe' will be a dictionary, as you originally intended.
for recipe in recipes:
    # Extract the title
    title = recipe['title']
    
    # Combine all ingredients into a single list
    all_ingredients = recipe['ingredients_from_cart'] + recipe['ingredients_to_add']
    
    # Format the ingredients list into a comma-separated string
    ingredients_list = [f"{ing['quantity']} {ing['item']}" for ing in all_ingredients]
    ingredients_string = ",".join(ingredients_list)
    
    # Join the instructions into a single block of text with newlines
    # Use replace('\\n', '\n') if you want actual newlines in the printout,
    # or keep '\\n' if you need the literal backslash-n for another system.
    instructions_string = "\\n".join(instruction.lstrip("0123456789. ") for instruction in recipe['instructions'])
    final_output = f"Title:{title},Ingredients:{ingredients_string},Instructions:{instructions_string}"
    
    # --- NEW: Add (append) the final string to our list ---
    processed_recipes_list.append(final_output)


In [48]:
contents = processed_recipes_list[0]

In [26]:
# contents = 'Title:Miso-Butter Roast Chicken With Acorn Squash Panzanella,Ingredients:1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher salt, divided, plus more, 2 small acorn squash (about 3 lb. total), 2 Tbsp. finely chopped sage, 1 Tbsp. finely chopped rosemary, 6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature, ¼ tsp. ground allspice, Pinch of crushed red pepper flakes, Freshly ground black pepper, ⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups), 2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces, 2 Tbsp. extra-virgin olive oil, ½ small red onion, thinly sliced, 3 Tbsp. apple cider vinegar, 1 Tbsp. white miso, ¼ cup all-purpose flour, 2 Tbsp. unsalted butter, room temperature, ¼ cup dry white wine, 2 cups unsalted chicken broth, 2 tsp. white miso, Kosher salt, freshly ground pepper,Instructions:Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet.\nCombine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.\nAdd bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.\nPlace onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.\nPlace a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.\nMeanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.\nUsing your fingers, mash flour and butter in a small bowl to combine.\nSet reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.\nServe chicken with gravy and squash panzanella alongside.'

## Vector Search Setup

Initialize the AI Platform and prepare for similarity search against the recipe index.

In [49]:
import os
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Similarity Search

Query the recipe index to find the most similar existing recipes to our generated recipe.

In [50]:
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    # index_endpoint_name=f"projects//locations//indexEndpoints/{INDEX_ENDPOINT_ID}"
    index_endpoint_name=f"projects/{PROJECT_NUMBER}/locations/{LOCATION}/indexEndpoints/{index_endpoint}" # @param
)

In [ ]:
# Generate embeddings for the processed recipe
response = client.models.embed_content(
    model="gemini-embedding-001",
    contents=[contents],
    config=EmbedContentConfig(
        output_dimensionality=3072,
    ),
)
embedding_vector = response.embeddings[0].values

### Results Analysis

Map the search results back to readable recipe content and display the most similar recipes.

In [66]:
# Test query
response_ = my_index_endpoint.match(
    deployed_index_id="recipe_index",
    queries=[embedding_vector],
    num_neighbors=3,
)

response_

[[MatchNeighbor(id='772', distance=0.768811047077179, sparse_distance=None, feature_vector=None, crowding_tag=None, restricts=None, numeric_restricts=None, sparse_embedding_values=None, sparse_embedding_dimensions=None),
  MatchNeighbor(id='417', distance=0.7565523386001587, sparse_distance=None, feature_vector=None, crowding_tag=None, restricts=None, numeric_restricts=None, sparse_embedding_values=None, sparse_embedding_dimensions=None),
  MatchNeighbor(id='688', distance=0.7508333921432495, sparse_distance=None, feature_vector=None, crowding_tag=None, restricts=None, numeric_restricts=None, sparse_embedding_values=None, sparse_embedding_dimensions=None)]]

In [67]:
df = pd.read_csv("recipes_embeddings.csv") # @param from previous notebook
df

,id,content,embedding,embedding_dim
0,0,Title:Miso-Butter Roast Chicken With Acorn Squ...,"[-0.0023760846816003323, -0.011549703776836395...",3072
1,1,"Title:Crispy Salt and Pepper Potatoes,Ingredie...","[0.01674710586667061, 0.010629256255924702, 0....",3072
2,2,"Title:Thanksgiving Mac and Cheese,Ingredients:...","[-0.0013591762399300933, -0.002013501012697816...",3072
3,3,"Title:Italian Sausage and Bread Stuffing,Ingre...","[0.006325311027467251, -0.005718541797250509, ...",3072
4,4,"Title:Newton's Law,Ingredients:1 teaspoon dark...","[0.011500250548124313, -0.005415198393166065, ...",3072
...,...,...,...,...
995,995,"Title:Winter of Our Content,Ingredients:3/4 ou...","[-0.018302129581570625, -0.004354100674390793,...",3072
996,996,"Title:Perfect Circle,Ingredients:1 1/2 cups ch...","[-0.02723771147429943, 0.004735851660370827, 0...",3072
997,997,"Title:All She Wrote,Ingredients:2 1/4 cups chi...","[-0.006035564001649618, 0.003681536763906479, ...",3072
998,998,"Title:Mr. Tingles' Punch,Ingredients:1 (750 ml...","[0.005597192794084549, 0.0076141138561069965, ...",3072


In [77]:
df['id'] = df['id'].astype(str)

df_lookup = df.set_index('id')

# Iterate through the response and map IDs back to content
matched_recipes = []
if response_ and response_[0]:
    for neighbor in response_[0]:
        try:
            # Now, the string `neighbor.id` will match the string index in `df_lookup`
            matched_content = df_lookup.loc[neighbor.id]["content"]
            
            matched_recipes.append({
                "ID": neighbor.id,
                "Content": matched_content,
                "Similarity": neighbor.distance
            })
        except KeyError:
            print(f"Warning: ID '{neighbor.id}' from search result not found in DataFrame.")

# Display the final, readable results
if matched_recipes:
    matched_df = pd.DataFrame(matched_recipes)
    print("\n--- Readable Search Results ---")
else:
    print("\nSomething went wrong, no matches were mapped.")

    
titles = matched_df['Content'].str.split(',Ingredients:', expand=True)[0].str.split('Title:', expand=True)[1].str.strip()
for title in titles:
    print(title)


--- Readable Search Results ---
Tandoori Chicken and Vegetable Sheet-Pan Supper
Crispy Sheet-Pan Broccoli
Chicken with Lemon and Spicy Spring Onions
